In [59]:
import glob
import os
import torch
import numpy as np
import xarray as xr
import netCDF4 as nc
import pandas as pd
import simpy as sp
import logging
import multiprocessing as mp
from multiprocessing import Pool

In [2]:
import cdsapi

In [3]:
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [4]:
os.chdir('..')

In [5]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datasets.km_dataset import DeepKoopmanDataset
from models.km_classical import ClassicalKoopmanAnalysis
from models.km_havoc import HAVOCAnalysis
from models.km_net import DeepKoopmanWrapper

In [7]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Koopman Forecast of Maritime Weather for Cargo Ship Route Optimization

## ERA5 Around Hamburg Harbor

ERA5 is a dataset which provides hourly maritime weather data.<br>
Its API does only provide single point measurements, but we can play a little and grab this data for multiple points all together.

![Map with coordinated of HH harbor with midpoint of the grid.](../img/map_grid_hhh.png)<br>
*Coastal map. The blue point is at coordinates lon 10 and lat 53.5, around Hamburg Harbor.*<br>
*The red dot is representing the initial radius taken when data mining the region of interest.*<br>
*One can see that that the mean on that point needs to be far more at the caostal line to get sufficiant forecasting!*

In [ ]:
# params
data_path = './data'
center_lon, center_lat = 5.5, 56.5
radius = 4.5
step = 0.5
start_date = '2025-02-01'
end_date = '2025-09-30'
parquet_path = os.path.join(data_path, f'{start_date}--{end_date}_hourly_era5.parquet')

In [ ]:
df = pd.read_parquet(parquet_path)
df

,lat,lon,u100,v100,u10,v10,d2m,t2m,msl,sst,...,ssrd,strd,tp,mwd,mwp,swh,miss,tp_zero,hour,date
2,52.0,2.0,-1.182327,6.403168,-1.083862,5.754166,276.31450,279.34050,103046.94,281.59595,...,0.0,926793.60,0.0,185.69049,4.595823,0.778139,False,True,0,2025-02-01
3,52.0,2.5,-1.313187,6.841644,-1.243042,6.271744,275.97858,279.32490,103074.69,281.68970,...,0.0,919625.60,0.0,228.72955,5.153929,0.901918,False,True,0,2025-02-01
4,52.0,3.0,-1.309280,6.379730,-1.294800,5.887955,276.23444,279.09247,103109.69,281.09497,...,0.0,926057.60,0.0,278.97955,5.439573,0.874330,False,True,0,2025-02-01
5,52.0,3.5,-1.298538,5.904144,-1.464722,5.008072,276.45320,278.36786,103147.19,279.98267,...,0.0,929801.60,0.0,308.49518,5.769896,0.783510,False,True,0,2025-02-01
6,52.0,4.0,-0.907913,5.661957,-1.045776,3.383072,275.20320,275.96942,103178.69,279.00317,...,0.0,920073.60,0.0,306.04987,5.658079,0.550844,False,True,0,2025-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096673,61.0,3.0,-4.056305,16.806274,-3.350845,13.281235,281.94867,286.12335,102681.56,285.94800,...,0.0,1206508.50,0.0,170.92366,6.642595,3.018293,False,True,23,2025-09-30
2096674,61.0,3.5,-3.455719,16.899048,-2.919205,13.669907,281.49164,286.16240,102763.06,286.35913,...,0.0,1152044.50,0.0,175.37679,6.575213,2.971662,False,True,23,2025-09-30
2096675,61.0,4.0,-2.604157,16.504517,-2.247330,13.646469,280.99164,286.32062,102839.06,287.30835,...,0.0,1095500.50,0.0,180.35335,6.412615,2.763655,False,True,23,2025-09-30
2096676,61.0,4.5,-1.921539,14.758423,-1.735611,12.393539,280.51117,286.39484,102910.31,287.67358,...,0.0,1050028.50,0.0,186.72054,6.042986,2.270491,False,True,23,2025-09-30


## Classical Koopman Analysis

When looking into stadard Fourier Analysis, we describe a linear system between the actual oscillators and the desired output space, minimizing the squared error:

$\begin{equation}E\left(K, \omega\right) = \sum_{t=0}^{T-1} \left(x - K \Omega(\omega t) \right)^2\end{equation}$

With $E$ being the actual error function between our ground truth $x$ and the oscillator $\Omega(\omega', t)$.


The oscillator $\Omega(\omega t)$ is defined in the same manner as in the Fourier wavelets:

$
\begin{equation}
\Omega(\omega t) = \begin{pmatrix} 
    \sin(\omega_1 t) \\
    \vdots \\
    \sin(\omega_N t) \\
    \cos(\omega_1 t) \\
    \vdots \\
    \cos(\omega_N t)
\end{pmatrix}
\end{equation}
$

In Koopman Analysis, however, we look into any type of function $f_{\Theta}$, mostly non-linear or at least quasi-linear:

$\begin{equation}E\left(f_{\Theta'}(\Omega(\omega', t)) | x, \Theta', \omega'\right) = \sum_{t=0}^{T-1} \left(x - f_{\Theta}(\Omega(\omega' t), \Theta') \right)^2\end{equation}$

Furthermore, from this error function, we can derive some type of pseudo $log$-likelihood, using any kind of error function for oscillators and periodic frequency elements:

$\begin{equation}\log L\left(\Theta', \omega'\right) = - E\left(f_{\Theta'}(\Omega(\omega' t)) | x, \Theta', \omega'\right) \end{equation}$

In this case we optimize our non-liearity $\Theta' \rightarrow \Theta$ and our frequencies $\omega' \rightarrow \omega$.

Here for such a pseudo-likelihood, we would use a softmax function over all samples $n$ and for all target dimensions $d$ to guarantee a distribution:

$\begin{equation}L\left(\Theta', \omega'\right) = \frac{\exp\left(\log L_{n, d}\left(\Theta', \omega'\right)\right)}
{\sum_{n=0}^{N-1} \sum_{\forall d} \exp\left(\log L_{n, d}\left(\Theta, \omega'\right)\right)} \end{equation}$




## HAVOC